### 인공신경망(ANN)

- 인공신경망 : 생물학적 뉴런에서 영감을 받아 만든 머신러닝 알고리즘, 신경망은 기존의 머신러닝 알고리즘으로 다루기 어려웠던 이미지, 음성, 텍스트 분야에서 뛰어난 성능을 발휘하면서 
크게 주목을 받고 있으며 종종 딥러닝이라고 부름

- 밀집층(dense layer, or 완전연결층(fully connected layer)) : 케라스의 레이어 안의 가장 기본이 되는 층으로 양쪽의 뉴런이 모두 연결되어 있음

- 텐서플로 : 구글이 만든 딥러닝 라이브러리로 CPU와 GPU를 이용해 인공신경망 모델을 효율적으로 훈련하며 모델 구축과 서비스에 필요한 다양한 도구 제공, 텐서플로 2.0부터는 신경망 모델을 빠르게 구성할 수 있는 케라스를 핵심 api로 채택. 케라스를 사용하면 간단한 모델에서 아주 복잡한 모델까지 손쉽게 만들 수 있음

- 원-핫 인코딩 : 타깃값을 해당 클래스만 1이고 나머지는 모두 0인 배열로 만드는 것, 다중 분류에서 크로스 엔트로피 손실 함수를 사용하려면 0,1,2와 같이 정수로 된 타깃값을 인코딩으로 변환



In [2]:
# 라이브러리 호출

import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier


In [3]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [4]:
# 훈련 데이터 shape
print(train_input.shape, train_target.shape)
# 테스트 데이터 shape
print(test_input.shape, test_target.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [5]:
train_scaled = train_input / 255.0 # 정규화
train_scaled = train_scaled.reshape(-1,28*28) # 1차원 배열 변환 

In [6]:
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size = 0.2, random_state=42)

In [7]:
print(train_scaled.shape, train_target.shape)
print(val_scaled.shape, val_target.shape)

(48000, 784) (48000,)
(12000, 784) (12000,)


### 밀집층(양쪽의 뉴런이 모두 연결하고 있기 때문에 완전연결층)

In [8]:
dense = keras.layers.Dense(10, activation = 'softmax', input_shape = (784,))
model = keras.Sequential(dense)

- 케라스의 Dense 클래스를 사용해 밀집층 마들기 - 필요한 매개변수는 뉴런 개수, 뉴런의 출력적용 함수, 입력크기
- 뉴런 개수는 10개인 패션아이템을 분류하기 때문에 10개, 적용 함수는 softmax 적용(이진 분류는 sigmoid 적용), 784개의 픽셀값 받음
- Sequential 클래스의 객체를 만들 때 앞에서 만든 밀집층의 객체 dense를 전달함, 여기서 만든 모델이 신경망 모델

- 은닉층에 활성화함수를 적용하는 이유 : 은닉층이 활성화함수가 없이 x와 가중치의 곱으로만 표현되어 있다면 뒤의 출력층과 합쳐서 하나의 층으로 표현할 수 있을 것. 은닉층에 활성화 함수를 거쳐서 비선형 함수로 데이터를 변형해서 단순히 선형식이 합쳐지지 못하도록 해야한다.

In [9]:
model.compile(loss = 'sparse_categorical_crossentropy', metrics='accuracy') # one-hot encoding으로 sparse하게 표현

- 이진 분류 : binary_crossentropy
- 다중 분류 : categorical_crossentropy
- 로지스틱 손실 함수는 다중 분류를 위한 손실 함수인 크로스 엔트로피(cross entropy) 손실 함수를 이진 분류 버전으로 만든 것

In [10]:
model.fit(train_scaled, train_target, epochs=10, batch_size=20)

Epoch 1/10
2400/2400 [==============================] - 8s 3ms/step - loss: 0.5939 - accuracy: 0.7978
Epoch 2/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.4840 - accuracy: 0.8408
Epoch 3/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.4654 - accuracy: 0.8481
Epoch 4/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.4562 - accuracy: 0.8529
Epoch 5/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.4506 - accuracy: 0.8538
Epoch 6/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.4460 - accuracy: 0.8571
Epoch 7/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.4435 - accuracy: 0.8583
Epoch 8/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.4429 - accuracy: 0.8585
Epoch 9/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.4371 - accuracy: 0.8611
Epoch 10/10
2400/2400 [==============================] - 8s 3ms/step - loss: 0.438

In [11]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 3ms/step - loss: 0.4661 - accuracy: 0.8516


[0.466084361076355, 0.8515833616256714]

### 하이퍼 파라미터 최적화

In [41]:
import keras_tuner as kt
import tensorflow as tf

In [47]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        hp_units1 = hp.Int('unit_1', min_value = 16, max_value = 180)
        hp_units2 = hp.Int('unit_2', min_value = 16, max_value = 180)
        hp_units4 = hp.Float('dropout_1', min_value = 0.0, max_value = 0.5, step = 0.05)
        hp_units5 = hp.Float('dropout_2', min_value = 0.0, max_value = 0.5, step = 0.05)
        
        model = keras.Sequential()
        model.add(layers.Dense(hp_units1, activation = 'relu', input_shape = [train_scaled.shape[1]]))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(hp_units4))
        model.add(layers.Dense(hp_units2, activation = 'relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(hp_units5))
        model.add(layers.Dense(9, activation = 'softmax'))
        
        hp_learning_rate = hp.Float('learning_rate', min_value = 1e-6, max_value = 1e-3, sampling = 'LOG')
        model.compile(optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
                      loss = 'sparse_categorical_crossentropy', metrics = ['Accuracy'])
        return model
    
        def fit(self, hp, model, *args, **kwargs):
            return model.fit(
                *args,
                batch_size = hp.Int('batch_size',min_value = 16, max_value = 256, step = 16),
                **kwargs
            )

In [48]:
tuner = kt.Hyperband(MyHyperModel(),
                     objective = 'val_accuracy',
                     max_epochs = 100,
                     executions_per_trial = 3,
                     overwrite = True,
                     factor = 3)

In [49]:
tuner.search(train_scaled, train_target, epochs = 10, validation_split = 0.1)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
45                |?                 |unit_1
136               |?                 |unit_2
0.05              |?                 |dropout_1
0.25              |?                 |dropout_2
1.7827e-05        |?                 |learning_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
4                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\liked\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\liked\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\liked\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\liked\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\liked\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\liked\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\liked\anaconda